In [2]:
import numpy as np
import PIL
from PIL import Image
from sklearn import datasets
from sklearn.model_selection import train_test_split

# 1. Download the Dataset and Understand the Format

In [ ]:
#Downloaded in the project directory

# 2. Generate the Data Matrix and the Label vector

In [7]:
i = 0
s = 1
x = 1
d = [0] * 400
y = [0] * 400

while i < 400:
  image = PIL.Image.open("dataset/s" + str(s) + "/" + str(x) + ".pgm")
  image_sequence = image.getdata()
  image_array = np.array(image_sequence)
  d[i] = image_array
  y[i] = s
  i = i + 1
  x = x + 1
  if x == 11:
    s = s + 1
    x = 1

400
10304


# 3. Split the Dataset into Training and Test sets

In [48]:
training_data = [0] * 200
testing_data = [0] * 200
training_labels = [0] * 200
testing_labels = [0] * 200
curr_index = 0

In [49]:
for i in range(1, 400, 2):
  curr_index = int(i / 2);
  testing_data[curr_index] = d[i]
  testing_labels[curr_index] = y[i]
  training_data[curr_index] = d[i - 1]
  training_labels[curr_index] = y[i - 1]

testing_data = np.array(testing_data)
testing_labels = np.array(testing_labels)
training_data = np.array(training_data)
training_labels = np.array(training_labels)

# 4. Classification using PCA

# 5. Classification Using LDA 

# 6. Classifier Tuning 

# 7. Compare vs Non-Face Images

# 8. Bonus 